In [86]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from mllib.utils.dvc import load_params
from mllib.utils.pkl import load_data_from_pkl
from PIL import Image
import torch

params = load_params()
model = load_data_from_pkl(Path(params['train']['model']))
test_dir = Path(params['data']['processed_test_dir'])
from src.train.dataset import get_grscl_transfrom

im_transform = get_grscl_transfrom()
output = []

for num, item in enumerate(test_dir.iterdir()):
    name = Path(item).stem
    im = Image.open(item)
    tr_im = im_transform(im).unsqueeze(dim=0)

    model.eval()
    with torch.inference_mode():
        tr_im_pred = model(tr_im)
    prd_label = int(torch.softmax(tr_im_pred, dim=1).argmax(dim=1))

    output_line = name + "\\" + str(prd_label)
    output.append(output_line)
    # DEBUG
    # if num == 2:
    #     break

output = "\n".join(output)
# name, prd_label
with open("predict/predict.txt", "w") as file:
    file.writelines(output)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
